In [1]:
!pip uninstall -y tf-nightly tensorflow
!pip install tf-nightly-2.0-preview --quiet
!pip install "tensorflow-hub>=0.3"




Uninstalling tensorflow-1.13.1:
  Successfully uninstalled tensorflow-1.13.1
     |████████████████████████████████| 87.9MB 1.2MB/s 
     |████████████████████████████████| 61kB 27.3MB/s 
     |████████████████████████████████| 430kB 51.4MB/s 
     |████████████████████████████████| 3.1MB 57.4MB/s 
ERROR: thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.


In [2]:
from __future__ import absolute_import, division, print_function

import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.0.0-dev20190519
Eager mode:  True
Hub version:  0.4.0
GPU is NOT AVAILABLE


In [3]:

import pandas as pd
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

import numpy as np
import re

from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder

from keras.utils import np_utils


arts = pd.read_csv("arts.csv")

sports = pd.read_csv("bleacher.csv")

business =  pd.read_csv("business.csv")

news = pd.read_csv("news.csv")

IAB1 = arts['Title'].tolist()

IAB17 = sports['Title'].tolist()

IAB12 = news['Title'].tolist()

IAB3 = business['Title'].tolist()

text = IAB1 + IAB17 + IAB12 + IAB3

labels = [0] * len(IAB1) + [1] * len(IAB17) + [2] * len(IAB12) + [3] * len(IAB3)





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [0]:



# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
encoded_labels = np_utils.to_categorical(encoded_Y)

np_text = np.array(text)

train_data, test_data, train_labels, test_labels = train_test_split(np_text, encoded_labels, test_size = 0.2, random_state = 1)


In [5]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data[:3])

<tf.Tensor: id=176, shape=(3, 20), dtype=float32, numpy=
array([[-0.31394356,  0.74191713,  0.9810596 ,  0.48566276, -0.19522838,
        -0.12403124,  0.33121604, -0.44481283, -1.4636467 , -0.35099757,
         0.3032147 ,  0.20566925, -0.553178  ,  0.37871677,  0.07298388,
         1.3021836 ,  0.7841976 ,  0.14137788,  0.2530746 ,  0.15806805],
       [-1.2470131 , -1.4582044 , -0.4146397 , -1.9365106 , -1.0359086 ,
         1.7920734 , -0.19926266, -0.54584336,  0.5554211 ,  1.0797051 ,
        -1.9896867 , -1.4887679 ,  0.5272539 , -0.6689655 ,  0.7111107 ,
        -1.144156  , -1.2670773 ,  0.77840006, -0.29270324,  0.75506073],
       [-0.36608362, -0.9092384 ,  0.4609631 , -1.6359096 ,  0.1481418 ,
         0.9011129 , -0.08865412, -0.07473722,  0.87411094,  0.5367575 ,
        -0.8755214 , -1.0592023 ,  0.18937157, -0.36306375,  0.75593865,
        -0.31274197, -1.1674571 ,  0.40116012,  0.82908493,  0.54220647]],
      dtype=float32)>

In [6]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 68        
Total params: 400,424
Trainable params: 400,424
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [8]:
history = model.fit(train_data,
                    train_labels,
                    epochs=40,
                    batch_size=512,
                    verbose=1)
                   

W0519 20:35:39.538178 140264670795648 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/40
35579/35579 [==============================] - 1s 20us/sample - loss: 1.3724 - acc: 0.3345
Epoch 2/40
35579/35579 [==============================] - 1s 15us/sample - loss: 1.1083 - acc: 0.5491
Epoch 3/40
35579/35579 [==============================] - 1s 16us/sample - loss: 0.9009 - acc: 0.6283
Epoch 4/40
35579/35579 [==============================] - 1s 14us/sample - loss: 0.7472 - acc: 0.7040
Epoch 5/40
35579/35579 [==============================] - 1s 14us/sample - loss: 0.6390 - acc: 0.7554
Epoch 6/40
35579/35579 [==============================] - 1s 14us/sample - loss: 0.5594 - acc: 0.7850
Epoch 7/40
35579/35579 [==============================] - 1s 14us/sample - loss: 0.5010 - acc: 0.8061
Epoch 8/40
35579/35579 [==============================] - 1s 14us/sample - loss: 0.4583 - acc: 0.8231
Epoch 9/40
35579/35579 [==============================] - 1s 14us/sample - loss: 0.4254 - acc: 0.8355
Epoch 10/40
35579/35579 [==============================] - 1s 14us/sample - loss: 

In [9]:
results = model.evaluate(test_data, test_labels)

print(results)

8895/8895 [==============================] - 0s 40us/sample - loss: 0.5199 - acc: 0.8259
[0.5199402126721816, 0.8258572]


In [12]:
text_labels = encoder.classes_

for i in range(400,600):
  print(test_data[i:i+1])
  print(text_labels[np.argmax(test_labels[i:i+1])])
  tweet_result = model.predict(test_data[i:i+1])
  print(text_labels[np.argmax(sum(tweet_result))])



["Trump And Chinese President Xi Jinping Are Reportedly 'Highly Unlikely' To Meet Before Critical Trade War Deadline"]
2
2
['Kim Kardashian and Kanye West back out of buying 14 million dollar home']
0
0
['Online reaction to Strasbourg attack']
2
2
['Kosovo votes to create an army, raising tensions with Serbia']
2
2
["Maroon 5 & Cardi B's 'Girls Like You' Holds No. 1 Spot, Lil Wayne Makes History | Billboard News"]
0
0
['Natalie Portman möchte eine andere Welt für ihre Kinder']
0
0
['Lady Gaga, Mark Ronson Talk "Shallow" Oscar Win Backstage | Oscars 2019']
0
0
['Rita Ora confident in her own skin']
0
0
["2018's Top Award Show Performances | Billboard News"]
0
0
["Wells Fargo's Taback Sees Tougher Fundraising Environment for Hedge Funds"]
3
3
['V-One, a nova boyband com um som contagiante']
0
1
["Steve Perry Discusses His Return to Music & New Album 'Traces' | Billboard"]
0
0
['Rare Star Wars Lightsaber Pulled From LA Auction']
2
2
["Trump: 'There Are A Lot Of Criminals In The Caravan'"]

In [21]:
news = ["Chuck Lorre says it's 'foolish' to think there won't be another 'Big Bang"]
tw_train = np.array(news)
tw_result = model.predict(tw_train)
value = text_labels[np.argmax(sum(tw_result))]

if value == 0:
  print("Entertainment")
elif value == 1:
  print("Sports")
elif value == 2:
  print("News")
elif value == 3:
  print("Business")

Entertainment
